In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models

In [ ]:
from torch.optim import lr_scheduler                                                                                                                                                                                       
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt
import numpy as np
import time
import torch.nn.functional as F
import os
from PIL import Image
import torchvision.transforms as transforms

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=6):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

In [ ]:
import argparse

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 超参数设置
EPOCH = 10   #遍历数据集次数
pre_epoch = 0  # 定义已经遍历数据集的次数
BATCH_SIZE = 4      #批处理尺寸(batch_size)
LR = 0.01        #学习率
model_use  = "aoi_model" 

import pandas as pd
data = pd.read_csv("./aoi/train.csv")

FullPath = os.getcwd()
data_dir  = os.path.join(FullPath + "/aoi/train_images")
model_dir = os.path.join(FullPath + "/models", model_use)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  #先四周填充0，在吧图像随机裁剪成32*32
    transforms.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5]), # normalize
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5]),
])

In [ ]:
data_images = []
for i in range(data.shape[0]):
    images_path = (os.path.join(data_dir + "/" +data.ID[i]))
    images = Image.open(images_path)
    images = transform_train(images)
    data_images.append(np.array(images))

In [ ]:
def split_data(data):
    tra = []
    #val = []
    for i, d in enumerate(data):
        #if i <= 2022:
        tra.append(d)
        #else:
        #    val.append(d)
    return tra

In [ ]:
tra_data = zip(data_images, np.array(data.Label))
tra = split_data(tra_data)

In [ ]:
tra_loader = DataLoader(tra,shuffle=True,batch_size = 4, num_workers = 2)
#val_loader = DataLoader(val,num_workers = 0,batch_size = 3)

In [ ]:
classes = ('0', '1', '2', '3', '4', '5')

In [ ]:
use_gpu = torch.cuda.is_available()
num_gpu = list(range(torch.cuda.device_count()))

# 模型定义-ResNet
net = ResNet18().to(device)

if use_gpu:
    ts = time.time()
    model = net.cuda()
    print("Finish cuda loading, time elapsed {}".format(time.time() - ts))
else:
#     nn.DataParallel(fcn_model)
    print("Use CPU to train.")

print(net)
params = list(net.parameters())

# 定义损失函数和优化方式
criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）

In [ ]:
def train():
    best_acc = 85  #2 初始化best test accuracy
    print("Start Training, Resnet-18!")  # 定义遍历数据集的次数
    
    with open("acc.txt", "w") as f:
        with open("log.txt", "w")as f2:
            for epoch in range(pre_epoch, EPOCH):
                configs    = "resnet_{}_batch{}_epoch{}_RMSprop_lr{}"\
                    .format(model_use, BATCH_SIZE, epoch, LR)
                model_path = os.path.join(model_dir, configs)
                print('\nEpoch: %d' % (epoch + 1))
                net.train()
                sum_loss = 0.0
                correct = 0.0
                total = 0.0
                for i, data in enumerate(tra_loader, 0):
                    # 准备数据
                    length = len(tra_loader)
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()

                    # forward + backward
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    # 每训练1个batch打印一次loss和准确率
                    sum_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += predicted.eq(labels.data).cpu().sum()
                    print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
                          % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
                    f2.write('%03d  %05d |Loss: %.03f | Acc: %.3f%% '
                          % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
                    f2.write('\n')
                    f2.flush()
                if epoch % 1 == 0:
                    torch.save(model.state_dict(),model_path + '.pkl')
            print("Training Finished, TotalEPOCH=%d" % EPOCH)

In [ ]:
#train()

In [ ]:
net = ResNet18().to(device)

test = pd.read_csv("./aoi/test.csv")
state_dict = torch.load(os.path.join(model_dir, "resnet_aoi_model_batch4_epoch9_RMSprop_lr0.01.pkl"), map_location='cpu')
model.load_state_dict(state_dict)
 
test_dir  = os.path.join(FullPath + "/aoi/test_images")
test_images = []
    
for i in range(test.shape[0]):
    images_path = (os.path.join(test_dir + "/" +test.ID[i]))
    images = Image.open(images_path)
    images = transform_test(images)
    test_images.append(np.array(images))
    
if use_gpu:
    ts = time.time()
    model = net.cuda()
    print("Finish cuda loading, time elapsed {}".format(time.time() - ts))
else:
#     nn.DataParallel(fcn_model)
    print("Use CPU to train.")


In [ ]:
test_loader = DataLoader(test_images,shuffle=False,batch_size = 4, num_workers = 2)

In [ ]:
result = []
for i,data in enumerate(test_loader):
    if use_gpu:
        inputs = Variable(data.cuda())
    else:
        inputs, labels = Variable(data)
    outputs = net(inputs)
    outputs = outputs.data.cpu().numpy()
    result.append(np.argmax(outputs, axis=1))